In [1]:
import cudf 
import cupy 

In [2]:
features = cudf.read_csv('/kaggle/input/ab-it-cup/preprocessed_feat_cup.csv')

In [3]:
target = cudf.read_csv('/kaggle/input/ab-it-cup/owner_target_final.csv')

In [4]:
y = (features.merge(target, how='left', left_on='client_id', right_on='owner_id')['owner_id'].fillna(0).astype(str) != '0').astype(int)

In [5]:
x = features.drop(['client_id', 'Unnamed: 0'], axis=1)

In [11]:
cat_features = x.columns[x.dtypes == object]

In [13]:
cat_features

Index(['dt', 'country', 'clientoutflowstatus', 'stratsegfactor', 'srvpackage',
       'clientsegment', 'gender', 'addrref',
       'hdb_relend_client_credits_count_grade'],
      dtype='object')

In [14]:
from cuml.preprocessing import OneHotEncoder

one_hot = OneHotEncoder()
x_cat = one_hot.fit_transform(x[cat_features])

In [30]:
X = cupy.hstack([x.drop(cat_features, axis=1).values, x_cat.todense()])

In [33]:
from cuml import KNeighborsClassifier

knn = KNeighborsClassifier(verbose=True)
knn.fit(X, y)

KNeighborsClassifier()

In [41]:
knn.predict_proba(X)

array([[1. , 0. ],
       [1. , 0. ],
       [1. , 0. ],
       ...,
       [0.6, 0.4],
       [0.8, 0.2],
       [1. , 0. ]], dtype=float32)

In [37]:
y.values

array([0, 0, 0, ..., 0, 0, 0])

In [45]:
from cuml.metrics import roc_auc_score

roc_auc_score(y_score=knn.predict_proba(X)[:, 1], y_true=y)

0.8927927613258362

In [74]:
from sklearn.model_selection import StratifiedGroupKFold

sgkf = StratifiedGroupKFold(n_splits=5)
folds = sgkf.split(X.tolist(), y.to_arrow().to_pylist(), features['client_id'].to_pandas())

In [75]:
roc_auc_metrics = []
num_fold = 1
for (train_index, test_index) in folds:
    print(f'FOLD {num_fold}/5')
    X_train_batch = X[train_index]
    X_test_batch = X[test_index]
    y_train_batch = y[train_index]
    y_test_batch = y[test_index]
    
    model = KNeighborsClassifier(verbose=True)

    model.fit(X_train_batch, y_train_batch)
    preds = model.predict_proba(X_test_batch)[:, 1]
    roc_auc_metrics.append(roc_auc_score(y_true=y[test_index], y_score=preds))
    num_fold += 1

FOLD 1/5
FOLD 2/5
FOLD 3/5
FOLD 4/5
FOLD 5/5


In [82]:
cupy.array(roc_auc_metrics).mean()

array(0.49967712)